## Text Classification Inference using Online Endpoints

This sample shows how to deploy `text-classification` type models to an online endpoint for inference.

### Task
`text-classification` is generic task type that can be used for scenarios such as sentiment analysis, emotion detection, grammar checking, spam filtering, etc. In this example, we will do sentiment analysis on movie reviews, to determine whether a review is positive or negative. 

### Inference data
We will use the [imdb](https://huggingface.co/datasets/imdb) dataset

### Model
Look for models tagged with `text-classification` in the system registry. Just looking for `text-classification` is not sufficient, you need to check if the model is specifically finetuned for sentiment analysis by studying the model card and looking at the input/output samples or signatures of the model. In this notebook, we use the `finiteautomata-bertweet-base-sentiment-analysis` model.

   

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Download and prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group_name="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
)
# The models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(credential, registry_name="azureml")

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `fill-mask` task. In this example, we use the `finiteautomata-bertweet-base-sentiment-analysis` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [ ]:
model_name = "roberta-large-mnli"
version_list = list(registry_ml_client.models.list(model_name))
if len(version_list) == 0:
    print("Model not found in registry")
else:
    model_version = version_list[0].version
    foundation_model = registry_ml_client.models.get(model_name, model_version)
    print(
        "\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(
            foundation_model.name, foundation_model.version, foundation_model.id
        )
    )

### 3. Download and prepare data for inference.

The next few cells show basic data preparation:
* Visualize some data rows
* Replace numerical categories in data with the actual string labels. This mapping is available in [label.json](./emotion-dataset/label.json). This step is needed because the selected models will return labels such `pos`, `neg`, etc. when running prediction. If the labels in your ground truth data are left as `0`, `1`, `2`, etc., then they would not match with prediction labels returned by the models.

In [ ]:
# Download a small sample of the dataset into the ./imdb-dataset directory
%run ./imdb-dataset/download-dataset.py --download_dir ./imdb-dataset

In [ ]:
import os

dataset_dir = "./imdb-dataset"
data_file = "train.jsonl"

# load the train.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd

pd.set_option(
    "display.max_colwidth", 0
)  # set the max column width to 0 to display the full text
df = pd.read_json(os.path.join(dataset_dir, data_file), lines=True)
df.head()

# load the id2label json element of the label.json file into pandas table with keys as 'label' column of int64 type and values as 'label_string' column as string type
import json

label_file = "label.json"
with open(os.path.join(dataset_dir, label_file)) as f:
    id2label = json.load(f)
    id2label = id2label["id2label"]
    label_df = pd.DataFrame.from_dict(
        id2label, orient="index", columns=["label_string"]
    )
    label_df["label"] = label_df.index.astype("int64")
    label_df = label_df[["label", "label_string"]]

# join the train, validation and test dataframes with the id2label dataframe to get the label_string column
df = df.merge(label_df, on="label", how="left")
# rename the label_string column to ground_truth_label
df = df.rename(columns={"label_string": "ground_truth_label"})

import re

df["text"] = df["text"].apply(lambda x: re.sub(r"<.*?>", "", x))

df.head()

### 4. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    ProbeSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "text-class-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", to detect positive or negative sentiment in movie reviews",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS4_v2",
    instance_count=2,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=1000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=1000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 5. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [ ]:
import json

score_file = "sample_score.json"
# pick 5 random rows
sample_df = df.sample(5)
# reset the index of sample_df
sample_df = sample_df.reset_index(drop=True)
sample_df.drop(columns=["label"], inplace=True)

# save the json object to a file named sample_score.json in the
test_json = {
    "input_data": sample_df["text"].tolist(),
    "params": {"return_all_scores": True},
}
# save the json object to a file named sample_score.json in the ./imdb-dataset folder
with open(os.path.join(".", dataset_dir, score_file), "w") as f:
    json.dump(test_json, f)
sample_df.head()

In [ ]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file=os.path.join(".", dataset_dir, score_file),
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
# rename label column to predicted_label
response_df = response_df.rename(columns={0: "predicted_label"})
response_df.head()

In [ ]:
# merge the sample_df and response_df dataframes
merged_df = sample_df.merge(response_df, left_index=True, right_index=True)
merged_df.head()

### 6. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()